## **Word2Vec Skip-gram**

In [10]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [11]:
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Embedding, Reshape, Conv1D, MaxPool1D, Dropout
from tensorflow.keras.preprocessing.text import one_hot,Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# **Read Data**

In [ ]:
data = pd.read_csv('data.txt')

In [ ]:
data.head()

,1
0,ای رستخیز ناگهان، وی رحمت بی منتها\tای آتشی اف...
1,امروز خندان آمدی، مفتاح زندان آمدی\tبر مستمندا...
2,خورشید را حاجب تویی، امید را واجب تویی\tمطلب ت...
3,در سینه ها برخاسته، اندیشه را آراسته\tهم خویش ...
4,ای روح بخش بی بَدَل، وی لذتِ علم و عمل\tباقی ب...


**Read Stop-Words**

In [ ]:
def read_stop_words(filename):
  with open(filename) as stopwords_file:
    stopwords = stopwords_file.readlines()
  stopwords = [line.replace('\n', '') for line in stopwords] 
  return stopwords

In [ ]:
stopwords = read_stop_words('stopwords.txt')
print(len(stopwords))

1382


**hazm library**

In [ ]:
# install hazm library
!pip install hazm
from hazm import word_tokenize

     |████████████████████████████████| 316 kB 5.4 MB/s 
     |████████████████████████████████| 233 kB 41.2 MB/s 
     |████████████████████████████████| 1.4 MB 48.2 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394485 sha256=52d7ccd818418ca1e57a67ba31a2f8cb79bddba42ea2008b81772532b59d1667
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154474 sha256=cc24d8f6220fe38c34fa1490dd3e255d8086ef8d21173c895f111d1596eb9761
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


### **Preprocess the text**

In [ ]:
# preprocess the text
def text_preprocess(data):
  text = [line.replace('\t', ' ') for line in data.values.flatten()]
  text = [line.replace('-', ' ') for line in text]
  text = [re.sub("\d+", "", t) for t in text]

  word_tokenized = [word_tokenize(t) for t in text]
  word_tokenized_filtered = [[w for w in sentence if w not in stopwords] for sentence in word_tokenized]

  sentences = [' '.join(sentence) for sentence in word_tokenized_filtered]
  sentences = [sentence for sentence in sentences if sentence != '']

  return sentences


In [ ]:
sentences = text_preprocess(data)

In [ ]:
sentences[0]

'رستخیز رحمت منتها آتشی افروخته بیشه اندیشه'

### **get less frequente words**

In [ ]:
def get_all_sentences():
  all_sentences = ''
  sentences = text_preprocess(data)
  for sentence in sentences:
    all_sentences += sentence
    all_sentences += ' '

  return all_sentences  

In [ ]:
def get_word_freq(vocabularies):
  word_freq = []
  for vocab in vocabularies:
    word_freq.append(vocabularies.count(vocab))
  return word_freq  

In [ ]:
# get the words that frequentes less than 2 times in the corpus
def get_less_frequente_words():
  low_frequency_words = []
  all_sentences = get_all_sentences()
  vocabularies = all_sentences.split(' ')
  word_freq = get_word_freq(vocabularies)

  for i in range(len(word_freq)):
    if word_freq[i] < 2:
      low_frequency_words.append(vocabularies[i])

  return low_frequency_words

In [ ]:
less_frequente_words = get_less_frequente_words()

In [ ]:
print(len(less_frequente_words))

6366


## **remove less frequente words**

In [ ]:
# remove the words that frequentes less than 2 times in the corpus
def remove_less_frequente_words(less_frequente_words, sentences):
  sentences_tokenized = [word_tokenize(sentence) for sentence in sentences]
  sentences_tokenized_filtered = [[w for w in sentence if w not in less_frequente_words] for sentence in sentences_tokenized]
  corpus = [' '.join(sentence) for sentence in sentences_tokenized_filtered]
  corpus = [sentence for sentence in corpus if sentence != '']

  return corpus

In [ ]:
corpus = remove_less_frequente_words(less_frequente_words, sentences)

In [ ]:
corpus[0:10]

['رستخیز رحمت منتها آتشی افروخته بیشه اندیشه',
 'خندان آمدی مفتاح زندان آمدی آمدی بخشش فضل خدا',
 'خورشید حاجب تویی امید واجب تویی مطلب تویی طالب تویی منتها',
 'سینه اندیشه آراسته حاجت روا',
 'روح علم باقی بهانه دغل علت دوا',
 'دغل کژ گنه کین مست مست نان شوربا',
 'هل عقل هل نان نشاید ماجرا',
 'تدبیر رنگ افکنی روم زنگ افکنی جنگ افکنی یری',
 'پنهان گوش جان بهانه کسان جان رب زنان والله کیا',
 'خامش رفتم پای علم کاغذ بنه بشکن قلم ساقی درآمد الصلا']

## **Tokenizer**

In [ ]:
# tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

In [ ]:
with open('tokenizer.h5', 'wb') as f:
    pickle.dump(tokenizer, f)

In [ ]:
list(tokenizer.word_index.items())[0:10]

[('جان', 1),
 ('دل', 2),
 ('عشق', 3),
 ('آب', 4),
 ('چشم', 5),
 ('شب', 6),
 ('جهان', 7),
 ('شمس', 8),
 ('دست', 9),
 ('مست', 10)]

In [ ]:
 # find more stopwords by sorting the tokenizer word counts
#  word_count_sorted = dict(sorted(tokenizer.word_counts.items(), reverse=False, key=lambda t: t[1]))
#  word_count_sorted

In [ ]:
encoded = tokenizer.texts_to_sequences(corpus)

In [ ]:
encoded[0:10]

[[2726, 283, 1337, 397, 1610, 967, 143],
 [185, 568, 1124, 398, 568, 568, 830, 332, 23],
 [46, 2727, 29, 636, 2031, 29, 831, 29, 258, 29, 1337],
 [92, 143, 1338, 727, 728],
 [30, 221, 162, 399, 1611, 968, 244],
 [1611, 729, 2032, 569, 10, 10, 209, 2728],
 [464, 13, 464, 209, 1125, 419],
 [969, 104, 1612, 352, 1126, 1612, 192, 1612, 730],
 [77, 41, 1, 399, 1613, 1, 259, 245, 284, 1127],
 [201, 731, 56, 221, 2729, 377, 732, 1128, 31, 570, 186]]

In [ ]:
num_all_words = sum(len(s) for s in encoded) # total number of words in the corpus
num_unique_words = len(tokenizer.word_index) + 1  # total number of unique words in the corpus

In [ ]:
num_all_words, num_unique_words

(32654, 4284)

### **Generate data**

In [ ]:
# Parameters
window_size = 2

In [ ]:
def generate_data(corpus, window_size, num_unique_words):
    maxlen = window_size * 2
    all_inputs = []
    all_outputs = []
    for words in corpus:
      len_words = len(words)
      for index,w in enumerate(words):
        s = index - window_size
        e = index + window_size + 1
        for i in range(s, e):
            if i != index and 0 <= i < len_words:
              all_inputs.append(w) 
              #all_inputs.append(to_categorical(w, num_unique_words)) 
              all_outputs.append(to_categorical(words[i], num_unique_words))

    return (np.array(all_inputs), np.array(all_outputs))

In [ ]:
# Create training data
X_train, y_train = generate_data(encoded, window_size, num_unique_words)
X_train.shape, y_train.shape

((98832,), (98832, 4284))

In [ ]:
X_train, y_train

(array([2726, 2726,  283, ...,  619, 1810, 1810]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32))

In [ ]:
# for i in X_train:
#   print(np.where(i == 1))
print(np.where(X_train[0] == 1))

(array([], dtype=int64),)


## **Create Neural Network**

In [ ]:
embed_size=100
model = Sequential()
model.add(Embedding(input_dim=num_unique_words, output_dim=embed_size, input_length=1, embeddings_initializer='glorot_uniform'))
model.add(Reshape((embed_size, )))
model.add(Dense(num_unique_words, activation='softmax', kernel_initializer='glorot_uniform'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 100)            428400    
                                                                 
 reshape (Reshape)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4284)              432684    
                                                                 
Total params: 861,084
Trainable params: 861,084
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train, y_train, epochs=300, verbose=1, batch_size=1024)

Epoch 1/300
97/97 [==============================] - 12s 116ms/step - loss: 8.3305 - accuracy: 0.0185
Epoch 2/300
97/97 [==============================] - 11s 116ms/step - loss: 8.0868 - accuracy: 0.0278
Epoch 3/300
97/97 [==============================] - 11s 116ms/step - loss: 7.6407 - accuracy: 0.0274
Epoch 4/300
97/97 [==============================] - 11s 118ms/step - loss: 7.4582 - accuracy: 0.0292
Epoch 5/300
97/97 [==============================] - 11s 117ms/step - loss: 7.3597 - accuracy: 0.0310
Epoch 6/300
97/97 [==============================] - 11s 116ms/step - loss: 7.2707 - accuracy: 0.0345
Epoch 7/300
97/97 [==============================] - 11s 117ms/step - loss: 7.1752 - accuracy: 0.0381
Epoch 8/300
97/97 [==============================] - 11s 117ms/step - loss: 7.0710 - accuracy: 0.0421
Epoch 9/300
97/97 [==============================] - 11s 118ms/step - loss: 6.9593 - accuracy: 0.0458
Epoch 10/300
97/97 [==============================] - 11s 117ms/step - loss: 6.841

## **Save Model**

In [ ]:
model.save('model_skipgram.h5')

## **Load Model**

In [12]:
# laod model
model = load_model('model_skipgram.h5')

In [13]:
model

### **Get Most Similarity**

In [14]:
with open('tokenizer.h5', 'rb') as f:
    tokenizer = pickle.load(f)

In [40]:
def get_most_similarity(word, model=model, tokenizer=tokenizer, n=15):
  num_unique_words = len(tokenizer.word_index) + 1
  word_to_sequences = tokenizer.texts_to_sequences([word])[0]
  prediction = model.predict(word_to_sequences)[0]
  index = np.argsort(prediction)[::-1][:n]
  sequences_to_word = tokenizer.sequences_to_texts([index])[0]
  most_similarity = sequences_to_word.split(' ')
  return most_similarity

In [41]:
get_most_similarity('عشق')

['عشق',
 'جان',
 'دل',
 'عقل',
 'آتش',
 'جمله',
 'شاه',
 'شمس',
 'عاشقان',
 'مست',
 'دست',
 'جهان',
 'گشته',
 'عالم',
 'گذر']

In [42]:
get_most_similarity('شمس')

['تبریزی',
 'تبریز',
 'دین',
 'الدین',
 'الحق',
 'مفخر',
 'عشق',
 'جان',
 'خداوند',
 'نور',
 'الضحی',
 'دل',
 'مخدوم',
 'شمس',
 'دست']

In [43]:
get_most_similarity('کنج')

['عاشق',
 'ای',
 'خلا',
 'کنجی',
 'درآرد',
 'زندان',
 'نرسم',
 'نشین',
 'کنج',
 'خری',
 'درگاه',
 'بهاری',
 'متاع',
 'خندان',
 'طلب']

In [44]:
get_most_similarity('گل')

['خار',
 'آب',
 'گل',
 'بلبل',
 'باغ',
 'آتش',
 'یار',
 'ریحان',
 'دل',
 'چمن',
 'گلزار',
 'شکر',
 'زمین',
 'رعنا',
 'جان']

In [45]:
get_most_similarity('مست')

['مست',
 'چشم',
 'آمدست',
 'جان',
 'خاک',
 'عشق',
 'خراب',
 'عقل',
 'ره',
 'دست',
 'یار',
 'جام',
 'ماییم',
 'پست',
 'الست']

In [46]:
get_most_similarity('دست')

['دست',
 'جان',
 'دل',
 'پا',
 'پای',
 'عشق',
 'جام',
 'دهان',
 'مست',
 'شمس',
 'دلم',
 'شب',
 'باده',
 'کف',
 'ساغر']

In [47]:
get_most_similarity('عقل')

['عشق',
 'عقل',
 'جان',
 'مست',
 'خار',
 'دل',
 'گل',
 'بلا',
 'سلطان',
 'دین',
 'قضا',
 'روح',
 'خط',
 'جمله',
 'عقیله']

In [48]:
get_most_similarity('غم')

['غم',
 'دل',
 'جان',
 'شادی',
 'برو',
 'شاد',
 'جهان',
 'اندوه',
 'آب',
 'عشق',
 'باده',
 'رود',
 'دست',
 'لطف',
 'عاشق']

In [49]:
get_most_similarity('انگور')

['پخته',
 'باغبان',
 'غوره',
 'نفی',
 'شوره',
 'شد',
 'ای',
 'شور',
 'کدامین',
 'دل',
 'روضه',
 'مستم',
 'گور',
 'برده',
 'ساقیا']

In [50]:
get_most_similarity('آب')

['گل',
 'آب',
 'حیات',
 'سنگ',
 'جان',
 'جوی',
 'آتش',
 'بحر',
 'روان',
 'جو',
 'دل',
 'حیوان',
 'نان',
 'چشمه',
 'کوزه']

In [51]:
get_most_similarity('آتش')

['عشق',
 'آتش',
 'آب',
 'جان',
 'گل',
 'دل',
 'برآوردم',
 'مفرش',
 'آفتاب',
 'سوز',
 'خلیل',
 'فلک',
 'زدی',
 'جگر',
 'خانه']

In [52]:
get_most_similarity('شب')

['شب',
 'جان',
 'مه',
 'گشت',
 'سحر',
 'دل',
 'ماه',
 'دام',
 'خلق',
 'خواب',
 'تیره',
 'دست',
 'شمع',
 'سیاه',
 'رخ']

In [53]:
get_most_similarity('جهان')

['جان',
 'جهان',
 'دل',
 'عشق',
 'نهان',
 'جمله',
 'غم',
 'شاه',
 'گرد',
 'تنگ',
 'دید',
 'آب',
 'عاشق',
 'ذره',
 'کهی']

In [54]:
get_most_similarity('دل')

['جان',
 'دل',
 'عشق',
 'خانه',
 'غم',
 'دست',
 'خون',
 'جهان',
 'تبریز',
 'رود',
 'سبک',
 'تن',
 'دین',
 'چشم',
 'آتش']

In [55]:
get_most_similarity('مه')

['مه',
 'نور',
 'شب',
 'رخ',
 'خورشید',
 'فلک',
 'جان',
 'ماه',
 'لقایی',
 'لقا',
 'سجده',
 'شمس',
 'توبه',
 'نهان',
 'دیدن']

In [56]:
get_most_similarity('نور')

['مه',
 'شمع',
 'نور',
 'شمس',
 'نار',
 'دل',
 'شعله',
 'تبریزی',
 'عالم',
 'آفتاب',
 'دیدار',
 'سایه',
 'چشم',
 'صبح',
 'موسی']